# Word2Vec practice(Pytorch)

If you don't have data for word2vec, you can download the dataset
from https://wit3.fbk.eu/get.php?path=XML_releases/xml/ted_en-20160408.zip&filename=ted_en-20160408.zip,  
or you can download the dataset using urlib.request like following.

### import urlib.request  
urllib.request.urlretrieve("https://raw.githubusercontent.com/GaoleMeng/RNN-and-FFNN-textClassification/master/ted_en-20160408.xml", filename="ted_en-20160408.xml")

In [6]:
# Packages for preprocessing
import re
from lxml import etree

## Preprocess dataset  
I follow preprocessing .xml file in the following site.  
https://wikidocs.net/60855  
  
1. Load the dataset: open()  
2. Extract the contents between CONTENTS and /CONTENTS
3. Substitute not text element to ' '
4. Split the text into sentences.
5. Eiminate the punctuation marks and substitute it to blank
   & Change the capital letter to a small letter
6. Tokenize the preprocessed sentences

In [36]:
dataset = open('dataset/ted_en-20160408.xml', 'r', encoding='UTF8')

text = '\n'.join(etree.parse(dataset).xpath('//content/text()'))
text = re.sub(r'\([^)]*\)', ' ', text)
print("*Print one sentence in text:\n\n{}".format(text[:95]))

sentences = text.split('.')
print("\n*Print one sentence in sentences:\n\n{}".format(sentences[0]))

pre_sentences = []
for sentence in sentences:
    pre_sentences.append(re.sub(r"[^a-z0-9]+", " ", sentence.lower()))

print("\n*Print one sentence in pre_sentences:\n\n{}".format(pre_sentences[0]))

tokenized_sentence = [sentence.split(" ") for sentence in pre_sentences]
print("\n*Print one sentence in tokenized_sentence:\n\n{}".format(tokenized_sentence[0]))

*Print one sentence in text:

Here are two reasons companies fail: they only do more of the same, or they only do what's new.

*Print one sentence in sentences:

Here are two reasons companies fail: they only do more of the same, or they only do what's new

*Print one sentence in pre_sentences:

here are two reasons companies fail they only do more of the same or they only do what s new

*Print one sentence in tokenized_sentence:

['here', 'are', 'two', 'reasons', 'companies', 'fail', 'they', 'only', 'do', 'more', 'of', 'the', 'same', 'or', 'they', 'only', 'do', 'what', 's', 'new']


## Word to index & Index to Word
Now, we have to change a token to an one hot vector  
I follow the instruction from https://towardsdatascience.com/implementing-word2vec-in-pytorch-skip-gram-model-e6bae040d2fb    

1. Make vocabulary from tokenized_sentence
2. Count the word frequency and cut words that are appear less than min_freq
3. Create dictionaries for mapping between word and index 

In [ ]:
vocabulary = []
count = []
min_freq = 5

for sentence in tokenized_sentence:
    for token in sentence:
        if token not in vocabulary:
            vocabulary.append(token)
            count.append(1)
        else:
            count[vocabulary.index(token)] += 1

VOCAB = [vocabulary[i] for i, cnt in enumerate(count) if cnt >= min_freq]
SENTENCE = []

for sentence in tokenized_sentence:
    temp = [token for token in sentence if token in VOCAB]
    SENTENCE.append(temp)

word2index = {word: idx for (idx, word) in enumerate(VOCAB)}
index2word = {idx: word for (idx, word) in enumerate(VOCAB)}

vocab_size = len(VOCAB)